In [1]:
import numpy as np
import pandas as pd
import gc
import time

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

import warnings
from time import time
import xgboost as xgb
xgb.__version__

'1.6.0-dev'

In [2]:
%%time
train = pd.read_csv('../input/train_1830.csv')

CPU times: user 44.9 s, sys: 4.34 s, total: 49.2 s
Wall time: 51.5 s


In [3]:
application_train = pd.read_csv('../input/application_train.csv')
target = application_train.TARGET.values

In [4]:
good_features = train.columns[2:]

In [5]:
params = {'alpha': 0.10267123481648575,
          'colsample_bytree': 0.5581906611108047,
          'eval_metric': 'auc',
          'lambda': 0.4246900552568794,
          'learning_rate': 0.017056331528057946,
          'max_depth': 15,
          'min_child_weight': 96,
          'objective': 'binary:logistic',
          'subsample': 0.8453437746562742,
          'tree_method': 'gpu_hist'}

In [6]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [7]:
%%time
start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].values[train_index], train[good_features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=1000, evals=watchlist, 
                verbose_eval=500, 
                early_stopping_rounds = 500)
        
        #preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        #y_oof[test_index] = ypred
        #preds_total += preds
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        
        break

1
[0]	train-auc:0.76983	eval-auc:0.72185
[100]	train-auc:0.83528	eval-auc:0.77748
[200]	train-auc:0.84342	eval-auc:0.78223
[300]	train-auc:0.85803	eval-auc:0.78776
[400]	train-auc:0.87236	eval-auc:0.79169
[500]	train-auc:0.88437	eval-auc:0.79419
[600]	train-auc:0.89476	eval-auc:0.79555
[700]	train-auc:0.90397	eval-auc:0.79638
[800]	train-auc:0.91224	eval-auc:0.79696
[900]	train-auc:0.91975	eval-auc:0.79723
[999]	train-auc:0.92659	eval-auc:0.79749


/home/btunguz/.local/lib/python3.8/site-packages/xgboost/core.py:78: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 81.62638068199158
Fold AUC: 0.7975286768958999
CPU times: user 1min 57s, sys: 3.43 s, total: 2min
Wall time: 1min 21s


In [9]:
%%time
params['learning_rate'] = 0.001



start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].values[train_index], train[good_features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=200000, evals=watchlist, 
                verbose_eval=500, 
                early_stopping_rounds = 500)
        
        #preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        #y_oof[test_index] = ypred
        #preds_total += preds
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        
        break

1
[0]	train-auc:0.76984	eval-auc:0.72185
[500]	train-auc:0.82837	eval-auc:0.77385
[1000]	train-auc:0.83324	eval-auc:0.77536
[1500]	train-auc:0.83546	eval-auc:0.77684
[2000]	train-auc:0.83700	eval-auc:0.77827
[2500]	train-auc:0.83861	eval-auc:0.77957
[3000]	train-auc:0.84110	eval-auc:0.78109
[3500]	train-auc:0.84450	eval-auc:0.78280
[4000]	train-auc:0.84843	eval-auc:0.78447
[4500]	train-auc:0.85278	eval-auc:0.78615
[5000]	train-auc:0.85735	eval-auc:0.78779
[5500]	train-auc:0.86178	eval-auc:0.78926
[6000]	train-auc:0.86620	eval-auc:0.79063
[6500]	train-auc:0.87045	eval-auc:0.79179
[7000]	train-auc:0.87443	eval-auc:0.79271
[7500]	train-auc:0.87819	eval-auc:0.79347
[8000]	train-auc:0.88167	eval-auc:0.79415
[8500]	train-auc:0.88499	eval-auc:0.79473
[9000]	train-auc:0.88822	eval-auc:0.79521
[9500]	train-auc:0.89136	eval-auc:0.79565
[10000]	train-auc:0.89437	eval-auc:0.79603
[10500]	train-auc:0.89716	eval-auc:0.79635
[11000]	train-auc:0.89992	eval-auc:0.79665
[11500]	train-auc:0.90259	eval-au

/home/btunguz/.local/lib/python3.8/site-packages/xgboost/core.py:78: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 1596.120820760727
Fold AUC: 0.7987088134041834
CPU times: user 27min 35s, sys: 3.53 s, total: 27min 38s
Wall time: 26min 36s


In [10]:
%%time
params['learning_rate'] = 0.0001



start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].values[train_index], train[good_features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=2000000, evals=watchlist, 
                verbose_eval=2000, 
                early_stopping_rounds = 2000)
        
        #preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        #y_oof[test_index] = ypred
        #preds_total += preds
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        
        break

1
[0]	train-auc:0.76982	eval-auc:0.72182
[2000]	train-auc:0.82289	eval-auc:0.77257
[4000]	train-auc:0.82680	eval-auc:0.77369
[6000]	train-auc:0.82964	eval-auc:0.77449
[8000]	train-auc:0.83166	eval-auc:0.77509
[10000]	train-auc:0.83310	eval-auc:0.77559
[12000]	train-auc:0.83422	eval-auc:0.77615
[14000]	train-auc:0.83504	eval-auc:0.77672
[16000]	train-auc:0.83573	eval-auc:0.77732
[18000]	train-auc:0.83631	eval-auc:0.77786
[20000]	train-auc:0.83687	eval-auc:0.77836
[22000]	train-auc:0.83747	eval-auc:0.77886
[24000]	train-auc:0.83816	eval-auc:0.77938
[26000]	train-auc:0.83896	eval-auc:0.77991
[28000]	train-auc:0.83993	eval-auc:0.78051
[30000]	train-auc:0.84103	eval-auc:0.78113
[32000]	train-auc:0.84228	eval-auc:0.78178
[34000]	train-auc:0.84364	eval-auc:0.78243
[36000]	train-auc:0.84513	eval-auc:0.78312
[38000]	train-auc:0.84673	eval-auc:0.78382
[40000]	train-auc:0.84840	eval-auc:0.78451
[42000]	train-auc:0.85012	eval-auc:0.78519
[44000]	train-auc:0.85188	eval-auc:0.78586
[46000]	train-auc

/home/btunguz/.local/lib/python3.8/site-packages/xgboost/core.py:78: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 15799.227672815323
Fold AUC: 0.7987649921447728
CPU times: user 4h 23min 47s, sys: 17.7 s, total: 4h 24min 4s
Wall time: 4h 23min 19s


In [11]:
%%time
test = pd.read_csv('../input/test_1830.csv')

CPU times: user 7.34 s, sys: 196 ms, total: 7.54 s
Wall time: 7.87 s


In [13]:
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [14]:
xgtest = xgb.DMatrix(test[good_features].values)

In [15]:
y_oof = np.zeros((train.shape[0],))
preds_total = 0

In [16]:
%%time
params['learning_rate'] = 0.001

start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].values[train_index], train[good_features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=200000, evals=watchlist, 
                verbose_eval=500, 
                early_stopping_rounds = 500)
        
        preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        y_oof[test_index] = ypred
        preds_total += preds/5
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        

1
[0]	train-auc:0.76984	eval-auc:0.72185
[500]	train-auc:0.82837	eval-auc:0.77385
[1000]	train-auc:0.83324	eval-auc:0.77536
[1500]	train-auc:0.83546	eval-auc:0.77684
[2000]	train-auc:0.83700	eval-auc:0.77827
[2500]	train-auc:0.83861	eval-auc:0.77957
[3000]	train-auc:0.84110	eval-auc:0.78109
[3500]	train-auc:0.84450	eval-auc:0.78280
[4000]	train-auc:0.84843	eval-auc:0.78447
[4500]	train-auc:0.85278	eval-auc:0.78615
[5000]	train-auc:0.85735	eval-auc:0.78779
[5500]	train-auc:0.86178	eval-auc:0.78926
[6000]	train-auc:0.86620	eval-auc:0.79063
[6500]	train-auc:0.87045	eval-auc:0.79179
[7000]	train-auc:0.87443	eval-auc:0.79271
[7500]	train-auc:0.87819	eval-auc:0.79347
[8000]	train-auc:0.88167	eval-auc:0.79415
[8500]	train-auc:0.88499	eval-auc:0.79473
[9000]	train-auc:0.88822	eval-auc:0.79521
[9500]	train-auc:0.89136	eval-auc:0.79565
[10000]	train-auc:0.89437	eval-auc:0.79603
[10500]	train-auc:0.89716	eval-auc:0.79635
[11000]	train-auc:0.89992	eval-auc:0.79665
[11500]	train-auc:0.90259	eval-au

/home/btunguz/.local/lib/python3.8/site-packages/xgboost/core.py:78: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 1611.9575431346893
Fold AUC: 0.7987088134041834
2
[0]	train-auc:0.76935	eval-auc:0.71703
[500]	train-auc:0.82899	eval-auc:0.77012
[1000]	train-auc:0.83366	eval-auc:0.77169
[1500]	train-auc:0.83567	eval-auc:0.77267
[2000]	train-auc:0.83729	eval-auc:0.77382
[2500]	train-auc:0.83908	eval-auc:0.77518
[3000]	train-auc:0.84167	eval-auc:0.77676
[3500]	train-auc:0.84503	eval-auc:0.77842
[4000]	train-auc:0.84894	eval-auc:0.78015
[4500]	train-auc:0.85336	eval-auc:0.78202
[5000]	train-auc:0.85790	eval-auc:0.78370
[5500]	train-auc:0.86232	eval-auc:0.78523
[6000]	train-auc:0.86668	eval-auc:0.78658
[6500]	train-auc:0.87078	eval-auc:0.78774
[7000]	train-auc:0.87458	eval-auc:0.78870
[7500]	train-auc:0.87815	eval-auc:0.78953
[8000]	train-auc:0.88156	eval-auc:0.79024
[8500]	train-auc:0.88476	eval-auc:0.79087
[9000]	train-auc:0.88787	eval-auc:0.79142
[9500]	train-auc:0.89082	eval-auc:0.79193
[10000]	train-auc:0.89370	eval-auc:0.79238
[10500]	train-auc:0.89652	eval-auc:0.79275
[11000]	train-

/home/btunguz/.local/lib/python3.8/site-packages/xgboost/core.py:78: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 3275.0594804286957
Fold AUC: 0.7958022481191679
3
[0]	train-auc:0.76814	eval-auc:0.72861
[500]	train-auc:0.82686	eval-auc:0.78559
[1000]	train-auc:0.83168	eval-auc:0.78707
[1500]	train-auc:0.83379	eval-auc:0.78812
[2000]	train-auc:0.83533	eval-auc:0.78903
[2500]	train-auc:0.83705	eval-auc:0.79008
[3000]	train-auc:0.83959	eval-auc:0.79159
[3500]	train-auc:0.84299	eval-auc:0.79328
[4000]	train-auc:0.84705	eval-auc:0.79507
[4500]	train-auc:0.85148	eval-auc:0.79676
[5000]	train-auc:0.85619	eval-auc:0.79830
[5500]	train-auc:0.86071	eval-auc:0.79961
[6000]	train-auc:0.86516	eval-auc:0.80084
[6500]	train-auc:0.86943	eval-auc:0.80185
[7000]	train-auc:0.87334	eval-auc:0.80269
[7500]	train-auc:0.87709	eval-auc:0.80337
[8000]	train-auc:0.88059	eval-auc:0.80397
[8500]	train-auc:0.88385	eval-auc:0.80449
[9000]	train-auc:0.88702	eval-auc:0.80489
[9500]	train-auc:0.89011	eval-auc:0.80529
[10000]	train-auc:0.89306	eval-auc:0.80561
[10500]	train-auc:0.89588	eval-auc:0.80590
[11000]	train-

...

Time elapsed: 4762.534660577774
Fold AUC: 0.8078738426890136
4
[0]	train-auc:0.76849	eval-auc:0.72621
[500]	train-auc:0.82783	eval-auc:0.77884
[1000]	train-auc:0.83260	eval-auc:0.78020
[1500]	train-auc:0.83462	eval-auc:0.78134
[2000]	train-auc:0.83613	eval-auc:0.78248
[2500]	train-auc:0.83785	eval-auc:0.78372
[3000]	train-auc:0.84050	eval-auc:0.78527
[3500]	train-auc:0.84395	eval-auc:0.78687
[4000]	train-auc:0.84792	eval-auc:0.78855
[4500]	train-auc:0.85230	eval-auc:0.79034
[5000]	train-auc:0.85692	eval-auc:0.79203
[5500]	train-auc:0.86136	eval-auc:0.79357
[6000]	train-auc:0.86582	eval-auc:0.79498
[6500]	train-auc:0.87009	eval-auc:0.79617
[7000]	train-auc:0.87407	eval-auc:0.79713
[7500]	train-auc:0.87780	eval-auc:0.79795
[8000]	train-auc:0.88135	eval-auc:0.79867
[8500]	train-auc:0.88475	eval-auc:0.79927
[9000]	train-auc:0.88802	eval-auc:0.79980
[9500]	train-auc:0.89113	eval-auc:0.80026
[10000]	train-auc:0.89409	eval-auc:0.80067
[10500]	train-auc:0.89697	eval-auc:0.80102
[11000]	train-a

/home/btunguz/.local/lib/python3.8/site-packages/xgboost/core.py:78: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 6495.508581876755
Fold AUC: 0.8042138576879696
5
[0]	train-auc:0.76738	eval-auc:0.73241
[500]	train-auc:0.82751	eval-auc:0.78197
[1000]	train-auc:0.83218	eval-auc:0.78362
[1500]	train-auc:0.83419	eval-auc:0.78481
[2000]	train-auc:0.83587	eval-auc:0.78604
[2500]	train-auc:0.83763	eval-auc:0.78735
[3000]	train-auc:0.84033	eval-auc:0.78882
[3500]	train-auc:0.84379	eval-auc:0.79041
[4000]	train-auc:0.84780	eval-auc:0.79192
[4500]	train-auc:0.85218	eval-auc:0.79352
[5000]	train-auc:0.85682	eval-auc:0.79503
[5500]	train-auc:0.86127	eval-auc:0.79626
[6000]	train-auc:0.86572	eval-auc:0.79742
[6500]	train-auc:0.86994	eval-auc:0.79840
[7000]	train-auc:0.87395	eval-auc:0.79921
[7500]	train-auc:0.87769	eval-auc:0.79990
[8000]	train-auc:0.88132	eval-auc:0.80050
[8500]	train-auc:0.88468	eval-auc:0.80096
[9000]	train-auc:0.88795	eval-auc:0.80136
[9500]	train-auc:0.89100	eval-auc:0.80169
[10000]	train-auc:0.89397	eval-auc:0.80198
[10500]	train-auc:0.89685	eval-auc:0.80223
[11000]	train-a

/home/btunguz/.local/lib/python3.8/site-packages/xgboost/core.py:78: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


Time elapsed: 7938.7318913936615
Fold AUC: 0.8036529473938775
CPU times: user 2h 31min 54s, sys: 14.4 s, total: 2h 32min 9s
Wall time: 2h 12min 18s


In [17]:
roc_auc_score(target, y_oof)

0.8019563918699371